In [2]:
import os
import traceback
import numpy as np
import time
#from cuda import cuda, nvrtc

from hyperbolicTSNE.util import find_last_embedding
from hyperbolicTSNE.visualization import plot_poincare, animate
from hyperbolicTSNE import load_data, Datasets, SequentialOptimizer, initialization, HyperbolicTSNE

data_home = "datasets"
log_path = "temp/poincaregpu/"  # path for saving embedding snapshots

only_animate = False
seed = 42
dataset = Datasets.PLANARIA  # the Datasets handler provides access to several data sets used throughout the repository
num_points = 20000  # we use a subset for demonstration purposes, full MNIST has N=70000
perp = 15  # we use a perplexity of 30 in this example

dataX, dataLabels, D, V, _ = load_data(
    dataset, 
    data_home=data_home, 
    random_state=seed, 
    to_return="X_labels_D_V",
    hd_params={"perplexity": perp}, 
    sample=num_points, 
    knn_method="hnswlib"  # we use an approximation of high-dimensional neighbors to speed up computations
)

print("Data loaded")

exaggeration_factor = 12  # Just like regular t-SNE, we use early exaggeration with a factor of 12
learning_rate = (dataX.shape[0] * 1) / (exaggeration_factor * 1000)  # We adjust the learning rate to the hyperbolic setting
ex_iterations = 250  # The embedder is to execute 250 iterations of early exaggeration, ...
main_iterations = 750  # ... followed by 750 iterations of non-exaggerated gradient descent.












# ============= RUNNING EXACT GPU =============







opt_config = dict(
    learning_rate_ex=learning_rate,  # learning rate during exaggeration
    learning_rate_main=learning_rate,  # learning rate main optimization 
    exaggeration=exaggeration_factor, 
    exaggeration_its=ex_iterations, 
    gradientDescent_its=main_iterations, 
    vanilla=False,  # if vanilla is set to true, regular gradient descent without any modifications is performed; for  vanilla set to false, the optimization makes use of momentum and gains
    momentum_ex=0.5,  # Set momentum during early exaggeration to 0.5
    momentum=0.8,  # Set momentum during non-exaggerated gradient descent to 0.8
    exact=False,  # To use the quad tree for acceleration (like Barnes-Hut in the Euclidean setting) or to evaluate the gradient exactly
    area_split=False,  # To build or not build the polar quad tree based on equal area splitting or - alternatively - on equal length splitting
    n_iter_check=10,  # Needed for early stopping criterion
    size_tol=0.999  # Size of the embedding to be used as early stopping criterion
)

opt_params = SequentialOptimizer.sequence_poincare(**opt_config)

print("Sequence defined")

# Start: configure logging
logging_dict = {
    "log_path": log_path
}
opt_params["logging_dict"] = logging_dict

log_path = opt_params["logging_dict"]["log_path"]
# Delete old log path
if os.path.exists(log_path) and not only_animate:
    import shutil
    shutil.rmtree(log_path)
# End: logging

# Compute an initial embedding of the data via PCA
X_embedded = initialization(
    n_samples=dataX.shape[0],
    n_components=2,
    X=dataX,
    random_state=seed,
    method="pca"
)
print(X_embedded)

# Initialize the embedder
htsne = HyperbolicTSNE(
    init=X_embedded, 
    n_components=2, 
    metric="precomputed", 
    verbose=True, 
    opt_method=SequentialOptimizer, 
    opt_params=opt_params
)

X_embedded = initialization(
    n_samples=dataX.shape[0],
    n_components=2,
    X=dataX,
    random_state=seed,
    method="pca"
)
print(X_embedded)

start_time = time.time()

try:
    hyperbolicEmbedding = htsne.fit_transform((D, V))
except ValueError:
    print("Error!")
    hyperbolicEmbedding = find_last_embedding(log_path)
    traceback.print_exc()

end_time = time.time()

execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

# Create a rendering of the embedding and save it to a file
if not os.path.exists("results"):
    os.mkdir("results")
fig = plot_poincare(hyperbolicEmbedding, dataLabels)
fig.show()
fig.savefig(f"results/{dataset.name}-inexact.png")









# ============= RUNNING EXACT CPU =============






opt_config = dict(
    learning_rate_ex=learning_rate,  # learning rate during exaggeration
    learning_rate_main=learning_rate,  # learning rate main optimization 
    exaggeration=exaggeration_factor, 
    exaggeration_its=ex_iterations, 
    gradientDescent_its=main_iterations, 
    vanilla=False,  # if vanilla is set to true, regular gradient descent without any modifications is performed; for  vanilla set to false, the optimization makes use of momentum and gains
    momentum_ex=0.5,  # Set momentum during early exaggeration to 0.5
    momentum=0.8,  # Set momentum during non-exaggerated gradient descent to 0.8
    exact=True,  # To use the quad tree for acceleration (like Barnes-Hut in the Euclidean setting) or to evaluate the gradient exactly
    area_split=False,  # To build or not build the polar quad tree based on equal area splitting or - alternatively - on equal length splitting
    n_iter_check=10,  # Needed for early stopping criterion
    size_tol=0.999  # Size of the embedding to be used as early stopping criterion
)

opt_params = SequentialOptimizer.sequence_poincare(**opt_config)

print("Sequence defined")

log_path = "temp/poincarecpu/"  # path for saving embedding snapshots

# Start: configure logging
logging_dict = {
    "log_path": log_path
}
opt_params["logging_dict"] = logging_dict

log_path = opt_params["logging_dict"]["log_path"]
# Delete old log path
if os.path.exists(log_path) and not only_animate:
    import shutil
    shutil.rmtree(log_path)
# End: logging

# Compute an initial embedding of the data via PCA
X_embedded = initialization(
    n_samples=dataX.shape[0],
    n_components=2,
    X=dataX,
    random_state=seed,
    method="pca"
)

# Initialize the embedder
htsne = HyperbolicTSNE(
    init=X_embedded, 
    n_components=2, 
    metric="precomputed", 
    verbose=True, 
    opt_method=SequentialOptimizer, 
    opt_params=opt_params
)

start_time = time.time()

try:
    #hyperbolicEmbedding = htsne.fit_transform((D, V))
    print("would run exact")
except ValueError:
    print("Error!")
    hyperbolicEmbedding = find_last_embedding(log_path)
    traceback.print_exc()

end_time = time.time()

execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

# Create a rendering of the embedding and save it to a file
if not os.path.exists("results"):
    os.mkdir("results")
fig = plot_poincare(hyperbolicEmbedding, dataLabels)
fig.show()
fig.savefig(f"results/{dataset.name}-exact.png")

Data loaded
Please note that `empty_sequence` uses the KL divergence with Barnes-Hut approximation (angle=0.5) by default.
Sequence defined
[[-2.0058984e-05  7.1462935e-05]
 [-1.5478807e-05  2.9428158e-05]
 [-8.8770736e-05  1.9175363e-04]
 ...
 [-4.2159012e-05 -1.2194294e-05]
 [-2.6598855e-05 -5.1154544e-05]
 [-1.0723040e-05 -4.6191377e-05]]
[[-2.0058984e-05  7.1462935e-05]
 [-1.5478807e-05  2.9428158e-05]
 [-8.8770736e-05  1.9175363e-04]
 ...
 [-4.2159012e-05 -1.2194294e-05]
 [-2.6598855e-05 -5.1154544e-05]
 [-1.0723040e-05 -4.6191377e-05]]
[HyperbolicTSNE] Received iterable as input. It should have len=2 and contain (D=None, V=None)
[hd_mat] Warning: There is nothing to do with given parameters. Returning given D and V
Running Gradient Descent, Verbosity: True
[gradient_descent] Warning!: because of logging, the cf will be computed at every iteration


Gradient Descent error: 110.33070 grad_norm: 3.04988e-01:   0%|          | 0/250 [00:00<?, ?it/s]

[GRID] start
[GRID] Init:  0.499267578125 ms
[GRID] Indices:  0.0 ms
[GRID] Counts:  0.0 ms
[GRID] Starts, counts:  0.0 ms
[GRID] Starts, counts:  0.0 ms
[GRID] Average positions:  35.441650390625 ms
[GRID] Max distances:  1.497314453125 ms
[GRID] new points:  0.0 ms
[UG] Grid generation:  0.03793835639953613 seconds
[UG] Mem alloc:  0.000499725341796875 seconds
[UG] Mem copy:  0.0009984970092773438 seconds
[UG] Run:  0.003993988037109375 seconds
[UG] Copy back:  0.01647329330444336 seconds
[UG] Total:  0.09734320640563965 seconds
[UG] neg_f:  0.098 s
[UG] pos_f:  0.078 s
[UG] applying forces:  0.02 s


Gradient Descent error: 110.33070 grad_norm: 3.04988e-01:   0%|          | 1/250 [00:00<01:23,  3.00it/s]

[GRID] start
[GRID] Init:  0.515869140625 ms
[GRID] Indices:  0.0 ms
[GRID] Counts:  0.4794921875 ms
[GRID] Starts, counts:  0.0 ms
[GRID] Starts, counts:  0.0 ms
[GRID] Average positions:  24.978271484375 ms
[GRID] Max distances:  0.99853515625 ms
[GRID] new points:  0.0 ms
[UG] Grid generation:  0.026972293853759766 seconds
[UG] Mem alloc:  0.0 seconds
[UG] Mem copy:  0.0009808540344238281 seconds
[UG] Run:  0.0 seconds
[UG] Copy back:  0.017969131469726562 seconds
[UG] Total:  0.045922279357910156 seconds
[UG] neg_f:  0.046 s
[UG] pos_f:  0.134 s
[UG] applying forces:  0.019 s


...

[GRID] start
[GRID] Init:  0.499267578125 ms
[GRID] Indices:  0.0 ms
[GRID] Counts:  0.0 ms
[GRID] Starts, counts:  0.0 ms
[GRID] Starts, counts:  0.0 ms
[GRID] Average positions:  25.458984375 ms
[GRID] Max distances:  1.497802734375 ms
[GRID] new points:  0.0 ms
[UG] Grid generation:  0.027456045150756836 seconds
[UG] Mem alloc:  0.0 seconds
[UG] Mem copy:  0.0009980201721191406 seconds
[UG] Run:  0.0 seconds
[UG] Copy back:  0.05041980743408203 seconds
[UG] Total:  0.07887387275695801 seconds
[UG] neg_f:  0.079 s
[UG] pos_f:  0.096 s
[UG] applying forces:  0.026000000000000002 s


Gradient Descent error: 110.07114 grad_norm: 6.26163e-01:  70%|███████   | 176/250 [01:12<00:26,  2.84it/s]

[GRID] start
[GRID] Init:  0.49853515625 ms
[GRID] Indices:  0.0 ms
[GRID] Counts:  0.0 ms
[GRID] Starts, counts:  0.0 ms
[GRID] Starts, counts:  0.0 ms
[GRID] Average positions:  43.430419921875 ms
[GRID] Max distances:  0.9990234375 ms
[GRID] new points:  0.0 ms
[UG] Grid generation:  0.045427560806274414 seconds
[UG] Mem alloc:  0.0 seconds
[UG] Mem copy:  0.0009980201721191406 seconds
[UG] Run:  0.0004982948303222656 seconds
[UG] Copy back:  0.053414344787597656 seconds
[UG] Total:  0.10033822059631348 seconds
[UG] neg_f:  0.101 s
[UG] pos_f:  0.088 s


Gradient Descent error: 110.02471 grad_norm: 6.24635e-01:  71%|███████   | 177/250 [01:13<00:26,  2.81it/s]

[UG] applying forces:  0.019 s
[GRID] start
[GRID] Init:  0.0 ms
[GRID] Indices:  0.0 ms
[GRID] Counts:  0.5 ms
[GRID] Starts, counts:  0.0 ms
[GRID] Starts, counts:  0.0 ms
[GRID] Average positions:  26.457275390625 ms
[GRID] Max distances:  0.9990234375 ms
[GRID] new points:  0.0 ms
[UG] Grid generation:  0.02845478057861328 seconds
[UG] Mem alloc:  0.0 seconds
[UG] Mem copy:  0.0004990100860595703 seconds
[UG] Run:  0.000499725341796875 seconds
[UG] Copy back:  0.016472339630126953 seconds
[UG] Total:  0.04592585563659668 seconds
[UG] neg_f:  0.046 s
